# Calculate robusteness from pi control runs

In [32]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [33]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [34]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol'):
    # 
    fin1 = fdir+'original ts/'+runs+'_sst.nc'
    fin2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp)) #/(y2-y1+1)) #freq.
        
    return nev  # sum(nev)

In [36]:
ny = 30
prc1 = 90
prc2 = 10

pinev=list()
for i in range(int(500/ny)):
    nev = num_coevents(i*ny, (i+1)*ny-1, prc1 , prc2)
    pinev.append(nev)
pinev,np.mean(pinev) 

([[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0,

In [37]:
#model by model

In [38]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.25  , 0.    , 0.1875, 0.25  , 0.125 , 0.125 , 0.3125, 0.25  ,
       0.0625, 0.1875, 0.4375, 0.375 , 0.3125, 0.    , 0.6875, 0.125 ,
       0.1875, 0.375 , 0.25  , 0.1875, 0.125 , 0.1875])

In [39]:
#16 30-yr periods MMEs

In [40]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.22727273, 0.5       , 0.13636364, 0.27272727, 0.18181818,
        0.18181818, 0.22727273, 0.18181818, 0.27272727, 0.27272727,
        0.22727273, 0.22727273, 0.18181818, 0.04545455, 0.31818182,
        0.18181818]),
 0.44829958403652315)

In [41]:
#MME mean and spread

In [42]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.22727272727272727, 0.09370694603676501)

In [43]:
#cross model mean and spread

In [44]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.22727272727272727, 0.15021936439093506)

In [45]:
## Calculate thresholds for PI Control runs

In [46]:
def get_thre(y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [47]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev, np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [48]:
def freq_difruns(ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'ssp585',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    return freq0,freq1,freq2

In [49]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [65]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

funev=list()
hinev=list()
test=list()
for i in range(int(500/ny)):
    pi,hi,fu = freq_difruns(ny,i*ny, (i+1)*ny-1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
np.mean(funev),np.mean(hinev),np.std(np.mean(hinev,axis=0))


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 0-29
0.2273 0.6696

historical: 1900-1929
1.0909 1.8807

historical 2: 1985-2014
5.2727 7.5208

future: 2070-2099
13.4091 10.4214
Percentiles: 90/10
piControl: 30-59
0.5 0.5

historical: 1900-1929
1.5 2.7757

historical 2: 1985-2014
4.5 7.1271

future: 2070-2099
12.2727 11.1158
Percentiles: 90/10
piControl: 60-89
0.1364 0.3432

historical: 1900-1929
1.1364 2.1169

historical 2: 1985-2014
4.8636 7.5576

future: 2070-2099
13.3636 11.344
Percentiles: 90/10
piControl: 90-119
0.2727 0.4454

historical: 1900-1929
1.5 2.8881

historical 2: 1985-2014
5.0909 7.7101

future: 2070-2099
14.4091 11.1096
Percentiles: 90/10
piControl: 120-149
0.1818 0.4896

historical: 1900-1929
1.1364 1.8901

historical 2: 1985-2014
4.8636 7.0472

future: 2070-2099
13.8636 10.3717
Percentiles: 90/10
piControl: 150-179
0.1818 0.3857

historical: 1900-1929
1.2273 2.4667

historical 2: 1985-2014
4.4545 7.5301

future: 2070-2099
12.727

(13.056818181818182, 4.9602272727272725, 7.158204761173891)

In [66]:
np.mean(test),np.mean(pinev)

(0.22727272727272727, 0.22727272727272727)

In [67]:
#model by model

In [68]:
fu_model_mean=np.mean(funev,axis=0)
fu_model_spread=np.std(funev,axis=0)
fu_model_mean

array([ 2.6875,  4.875 , 30.    , 20.625 ,  2.375 ,  1.125 , 17.9375,
       29.75  , 30.    , 30.    , 23.3125, 25.75  ,  7.25  ,  0.0625,
       12.125 ,  7.1875,  4.75  ,  6.    , 12.9375,  8.9375,  9.5625,
        0.    ])

In [69]:
np.mean(fu_model_mean),np.std(fu_model_mean)

(13.056818181818182, 10.605679443908448)

In [70]:
#mme mean and spread

In [71]:
fu_mme_mean=np.mean(funev,axis=1)
fu_mme_spread=np.std(funev,axis=1)
np.mean(fu_mme_spread)

11.157641923692989

In [72]:
fu_mme_mean,np.mean(fu_mme_mean),np.std(fu_mme_mean)

(array([13.40909091, 12.27272727, 13.36363636, 14.40909091, 13.86363636,
        12.72727273, 12.18181818, 13.        , 14.18181818, 12.59090909,
        11.77272727, 11.90909091, 13.5       , 12.90909091, 13.45454545,
        13.36363636]),
 13.056818181818182,
 0.7545488773197284)

In [73]:
#16 MMEs diff and spread

In [74]:
diff=list()
zip_obj=zip(fu_mme_mean,mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[13.181818181818182,
 11.772727272727273,
 13.227272727272727,
 14.136363636363635,
 13.681818181818182,
 12.545454545454545,
 11.954545454545455,
 12.818181818181818,
 13.909090909090908,
 12.318181818181818,
 11.545454545454547,
 11.681818181818182,
 13.318181818181818,
 12.863636363636363,
 13.136363636363637,
 13.181818181818182]

In [75]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(12.829545454545455, 0.7718076078084963)

In [76]:
#model by model diff and spread

In [77]:
diff=list()
zip_obj=zip(fu_model_mean,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[2.4375,
 4.875,
 29.8125,
 20.375,
 2.25,
 1.0,
 17.625,
 29.5,
 29.9375,
 29.8125,
 22.875,
 25.375,
 6.9375,
 0.0625,
 11.4375,
 7.0625,
 4.5625,
 5.625,
 12.6875,
 8.75,
 9.4375,
 -0.1875]

In [78]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(12.829545454545455, 10.573474359010909)